# Construct Master Dataset by linking different tables in MIMIC-IV-ED

Note:  MIMIC-ED should be downloaded at “data/ed" folder


## Prepare Python library and raw data

In [54]:
! pip install tqdm

Looking in indexes: http://mirrors.aliyun.com/pypi/simple

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [55]:
import argparse
from helpers import *
from medcode_utils import commorbidity, extract_icd_list

In [56]:
from dataset_path import mimic_iv_path, output_path

In [57]:
output_path = '/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/output'

In [58]:
print(output_path)

/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/output


In [59]:
complaint_dict = {"chiefcom_chest_pain" : "chest pain", "chiefcom_abdominal_pain" : "abdominal pain|abd pain", 
"chiefcom_headache" : "headache|lightheaded", "chiefcom_shortness_of_breath" : "breath", "chiefcom_back_pain" : "back pain", "chiefcom_cough" : "cough", 
"chiefcom_nausea_vomiting" : "nausea|vomit", "chiefcom_fever_chills" : "fever|chill", "chiefcom_syncope" :"syncope", "chiefcom_dizziness" : "dizz"}

## Defining health utilization timerange parameters in days
icu_transfer_timerange = 12 # hours
# past_ed_visits_timerange = 365
# past_admissions_timerange = 365 
# past_icu_visits_timerange = 365
next_ed_visit_timerange = 3
pd.set_option('display.max_columns', 100)


## Load raw data tables through pandas library

In [60]:
## Reading main tables
df_edstays = read_edstays_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/edstays.csv/edstays.csv")
df_patients = read_patients_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/patients.csv.gz")
df_admissions = read_admissions_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/admissions.csv.gz")
df_icustays = read_icustays_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/icustays.csv.gz")
df_triage = read_triage_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/triage.csv/triage.csv")
df_vitalsign = read_vitalsign_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/vitalsign.csv/vitalsign.csv")
df_pyxis = read_pyxis_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/pyxis.csv/pyxis.csv")
df_medrecon = read_pyxis_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/medrecon.csv/medrecon.csv")
df_diagnoses = read_diagnoses_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/diagnoses_icd.csv.gz")


In [61]:
#df_diagnoses_icd = read_diagnoses_table("/tmp/pycharm_project_211/MIMIC-IV-ED--CODE-main/dataset/diagnoses_icd.csv.gz")

## ED root table, demographic and outcomes

In [62]:
## Merging patients -> merging admissions -> merging triage -> master
df_master = merge_edstays_patients_on_subject(df_edstays ,df_patients)
df_master = merge_edstays_admissions_on_subject(df_master ,df_admissions)

In [63]:
## Adding age, mortality and ICU transfer outcome
df_master = add_age(df_master)
df_master = add_inhospital_mortality(df_master)
df_master = add_ed_los(df_master)
df_master = add_outcome_icu_transfer(df_master, df_icustays, icu_transfer_timerange)
df_master['outcome_hospitalization'] = ~pd.isnull(df_master['hadm_id'])
df_master['outcome_critical'] = df_master['outcome_inhospital_mortality'] | df_master[''.join(['outcome_icu_transfer_', str(icu_transfer_timerange), 'h'])]

# Sort Master table for further process
df_master = df_master.sort_values(['subject_id', 'intime']).reset_index()

# Filling subjects NA ethnicity, takes ~17s
df_master = fill_na_ethnicity(df_master)

## Health Utilization

In [64]:
## Generate past ED visits
df_master = generate_past_ed_visits(df_master, timerange=30)
df_master = generate_past_ed_visits(df_master, timerange=90)
df_master = generate_past_ed_visits(df_master, timerange=365)

In [65]:
## Oucome:  future ED revisit variables
df_master = generate_future_ed_visits(df_master, next_ed_visit_timerange)

In [66]:
## Generate past admissions
df_master = generate_past_admissions(df_master, df_admissions, timerange=30)
df_master = generate_past_admissions(df_master, df_admissions, timerange=90)
df_master = generate_past_admissions(df_master, df_admissions, timerange=365)

In [67]:
## Generate past icu visits
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=30)
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=90)
df_master  = generate_past_icu_visits(df_master, df_icustays, timerange=365)

In [68]:
## Generate numeric timedelta variables
df_master = generate_numeric_timedelta(df_master)

## Triage Information

In [69]:
## Mergining with triage table, Comment: revise the variable names? triage_*
df_master = merge_edstays_triage_on_subject(df_master, df_triage) ## note change to merge master 

In [70]:
## Encoding 10 chief complaints
df_master = encode_chief_complaints(df_master, complaint_dict)

## Comorbidities from diagnosis ICD

In [71]:
df_master = commorbidity(df_master, df_diagnoses, df_admissions, timerange = 356*5)

In [73]:
df_master.to_csv('df_master_with_comorbidity.csv', index=False)

In [74]:
# This function takes about 10 min

extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v9')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v9_3digit')
extract_icd_list(df_edstays, df_diagnoses, df_admissions, output_path, timerange = 356*5, version = 'v10')


Number of unique ICD codes v9:  8657
Number of unique ICD codes v9_3digit:  8598
Number of unique ICD codes v10:  14956


In [ ]:
print(df_admissions.columns)
print(df_diagnoses.columns) 
print(df_edstays.columns) 
print(df_medrecon.columns)
print(df_pyxis.columns)  

In [ ]:
#print(df_admissions.columns)

In [ ]:
#print(df_diagnoses.columns) 

In [ ]:
#print(df_admissions.head())

## ED Vital signs

In [75]:
df_master = merge_vitalsign_info_on_edstay(df_master, df_vitalsign, options=['last'])

## Medication

In [76]:
df_master = merge_med_count_on_edstay(df_master, df_pyxis)

In [77]:
df_master = merge_medrecon_count_on_edstay(df_master, df_medrecon)

## Review the master dataset and output

In [78]:
#df_master.head(100).to_csv(os.path.join(output_path, 'master_dataset_part.csv'), index=False)
# Full dataset:
df_master.to_csv(os.path.join(output_path, 'master_dataset_new.csv'), index=False)